## 売上資料Ⅵ
## 月別経費一覧　作成

- 作成年度の入力を忘れずに！

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 年度入力
nendo = 2021

# ファイル名指定
file_nm = "2021年度経費データ.csv"

In [ ]:
# 運送費 （棚卸廃棄損） 研修費 事務用品費 手数料 図書費 水道光熱/電気 租税公課 組合/諸会 賃貸料
# 通信費 備品・消耗品 広告宣伝費 旅費交通費 工具器具備品 保険料 修繕費 雑費 車両費 当座/UFJ東寺 試験研究費
index_nm = ['運送費','（棚卸廃棄損）','研修費','事務用品費','手数料','図書費','水道光熱/電気','租税公課','組合/諸会費','賃貸料',\
'通信費','備品・消耗品','広告宣伝費','旅費交通費','工具器具備品','保険料','修繕費','雑費','車両費','当座/UFJ東寺','試験研究費']

kamoku_cd_list = [852001, 0, 875, 846, 856, 849, 859001, 850, 848002, 855001, 845001, 847, 841, 842001, 216, 0, 0, 0, 0, 0, 872003]


In [ ]:
def month_set(nendo, month):
    """年と月で6桁の年月にする
    """
    if month < 6:
        this_month = int(str(nendo+1) + "0" + str(month))
    else:
        if month > 9:
            this_month = int(str(nendo) + str(month))
        else:
            this_month = int(str(nendo) + "0" + str(month))
    
    return this_month

In [ ]:
def mark_num_change(mark):
    """
        半角￥マークつきの金額部分の数値のみ取り出す
              例）販売価格 \824,529 --> 824529
    """
    if len(mark) >= 2:
        if mark[-2] == ".":
            mark = mark[:-2]
    # 「\」と「,」と数字以外の文字を消去する
    after = re.sub(r"\D", "", mark)
    # マイナス対応
    if "-" in mark:
        after = int(after) * (-1)
    else:
        after = int(after)
    return after

In [ ]:
# 12ヵ月分のlistを作成
list_12 = []
m = 6
y = month_set(nendo, m)
list_12.append(y)
for i in range(11):
    m += 1
    if m > 12:
        m = 1
    y = month_set(nendo, m)
    list_12.append(y)
print(list_12)

[202106, 202107, 202108, 202109, 202110, 202111, 202112, 202201, 202202, 202203, 202204, 202205]


In [ ]:
import numpy as np
import pandas as pd
import re

In [ ]:
target_col = ['伝票日付','借方','借方科目名','金額','摘要']
keihidf_i = pd.read_csv("/content/drive/MyDrive/Purchased_products/csv/" + file_nm, usecols=target_col, encoding="cp932")

In [ ]:
keihidf_i.head()

In [ ]:
def date_henkan(date):
    point_cnt = 0
    henkan_list = ['00','01','02','03','04','05','06','07','08','09','10','11','12']
    for i in range(len(date)):
        if point_cnt == 2:
            mm = henkan_list[mm]
            yyyymm = int(str(yyyy) + mm)
            break

        if (point_cnt == 1) and (date[i] == "."):
            mm = int(date[first_point:i])
            point_cnt += 1

        if (point_cnt == 0) and (date[i] == "."):
            ry = int(date[1:i])
            yyyy = ry + 2018
            first_point = i + 1
            point_cnt += 1
    return yyyymm

In [ ]:
# yyyy/mm/dd -> yyyy/mm
ym_list = []
for i in range(len(keihidf_i)):
    date_keihi = date_henkan(keihidf_i.iloc[i]['伝票日付'])
    ym_list.append(date_keihi)
keihidf_i['伝票月'] = ym_list

# 金額を数字に
keihidf_i = keihidf_i.fillna(0)
kingaku_list = []
for i in range(len(keihidf_i)):
    kingaku_list.append(mark_num_change(str(keihidf_i.iloc[i]['金額'])))
keihidf_i['金額'] = kingaku_list

In [ ]:
keihidf_i.head()

In [ ]:
keihidf_i['金額'] = keihidf_i['金額'].astype(int)
keihidf_i['伝票月'] = keihidf_i['伝票月'].astype(int)

In [ ]:
# dict 用意
set_dict = {}
for i in range(len(list_12)):
    set_dict[list_12[i]] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
# 月ごとの金額セット
for i in range(len(keihidf_i)):
    karikata_chk = keihidf_i.iloc[i]['借方']
    if (karikata_chk==859001) | (karikata_chk==859002) | (karikata_chk==859003):
        karikata_chk = 859001
    if karikata_chk in kamoku_cd_list:
        k = kamoku_cd_list.index(karikata_chk)
        set_dict[keihidf_i.iloc[i]['伝票月']][k] += keihidf_i.iloc[i]['金額']

In [ ]:
# DataFrame作成
df = pd.DataFrame(set_dict, index=index_nm)

In [ ]:
# Excelデータ作成
df.to_excel("/content/drive/MyDrive/Purchased_products/excel/月別経費一覧.xlsx", sheet_name='経費')